In [11]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings as CommunityHuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

load_dotenv()
llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.7)
embeddings = CommunityHuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "travel-assistant-index"
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [12]:
itinerary_prompt = ChatPromptTemplate.from_template("""
    Você é um assistente de viagens especializado em criar roteiros detalhados.
    Use o contexto abaixo para gerar um roteiro de viagem personalizado para o usuário.
    Contexto: {context}
    Pergunta do usuário: {question}
    Responda em formato de roteiro com dias, horários e sugestões de atividades.
    Roteiro:""")


itinerary_chain = ({
    "context": RunnableLambda(lambda x: vectorstore.as_retriever().invoke(x["question"])) | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
} | itinerary_prompt | llm | StrOutputParser())


In [13]:
logistics_prompt = ChatPromptTemplate.from_template("""
    Você é um assistente de viagens focado em logística e transporte.
    Responda à pergunta do usuário de forma direta, prática e objetiva.
    Pergunta do usuário: {question}
    Resposta:""")

logistics_chain = ({ "question": RunnablePassthrough() } | logistics_prompt | llm | StrOutputParser())

In [ ]:
local_info_prompt = ChatPromptTemplate.from_template("""
    Você é um especialista em informações locais e turismo.
    Use o contexto abaixo para responder à pergunta do usuário de forma clara e objetiva.
    Contexto: {context}
    Pergunta do usuário: {question}
    Responda de forma direta e concisa. Se a informação não estiver disponível, diga que não sabe.
    Informação:""")


local_info_chain = ({
    "context": RunnableLambda(lambda x: vectorstore.as_retriever().invoke(x["question"])) | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
} | local_info_prompt | llm | StrOutputParser())


In [15]:
translation_prompt = ChatPromptTemplate.from_template("""
    Você é um assistente de tradução e guia de frases úteis para viagens.
    Traduza a frase do usuário.
    Exemplo: "Onde fica o banheiro?" -> "Where is the bathroom?"
    Frase do usuário: {question}
    Tradução/Frase:""")

translation_chain = ({ "question": RunnablePassthrough() } | translation_prompt | llm | StrOutputParser())


In [ ]:
#Teste de Cadeia
print("Testando a cadeia de Roteiro...")
pergunta = "Crie um roteiro de 2 dias em Paris."
resposta = itinerary_chain.invoke({"question": pergunta})
print(f"Pergunta: {pergunta}\nResposta:\n{resposta}")
print("="*80)

Testando a cadeia de Roteiro...
Pergunta: Crie um roteiro de 2 dias em Paris.
Resposta:
**Roteiro de 2 dias em Paris**

**Dia 1**

* 9:00 - 10:00: **Torre Eiffel**
 Comece o dia visitando a Torre Eiffel, um dos ícones mais famosos da cidade. Pague a entrada e desfrute das vistas espetaculares da cidade desde a plataforma superior.
* 10:30 - 12:30: **Museu do Louvre**
 Dirija-se ao Museu do Louvre, um dos maiores museus do mundo. Visite a Mona Lisa e a Vênus de Milo, e explique-se por mais de 2 horas.
* 13:00 - 14:30: **Almoço**
 Pare para almoçar em um restaurante local, como o Le Bouillon Chartier. Escolha pratos tradicionais franceses, como coq au vin ou bouillabaisse.
* 15:00 - 16:30: **Champs-Élysées**
 Dirija-se à Champs-Élysées, a mais famosa avenida de Paris. Faça compras em lojas de luxo, como Louis Vuitton ou Chanel, ou assista a um filme em um dos cinemas da avenida.
* 17:30 - 19:00: **Arco do Triunfo**
 Visite o Arco do Triunfo, um monumento em homenagem às vitórias militare

In [17]:
print("Testando a cadeia de Logística...")
pergunta_logistica = "Como ir do aeroporto Charles de Gaulle para o centro de Paris?"
resposta_logistica = logistics_chain.invoke({"question": pergunta_logistica})
print(f"Pergunta: {pergunta_logistica}\nResposta:\n{resposta_logistica}")
print("="*80)



Testando a cadeia de Logística...
Pergunta: Como ir do aeroporto Charles de Gaulle para o centro de Paris?
Resposta:
Existem várias opções para se deslocar do aeroporto Charles de Gaulle (CDG) para o centro de Paris, dependendo do seu orçamento e preferências. Aqui estão algumas opções:

1. **Tren** (RER B): É uma opção rápida e econômica. Você pode pegar o trem na estação do aeroporto e ir até a estação Saint-Michel ou Gare du Nord, que estão próximas ao centro de Paris. O custo é de aproximadamente 10,50€.
2. **Metrô e RER**: Outra opção é pegar o metrô (linha 4) até a estação Marcadet-Poissonniers e, em seguida, mudar para a linha 4 do RER até a estação Saint-Michel ou Gare du Nord. Esse trajeto é um pouco mais lento, mas pode ser mais barato (cerca de 6€).
3. **Taxi**: É uma opção mais rápida e conveniente, mas mais cara (cerca de 60-80€). No entanto, se você estiver com muitas malas ou preferir uma opção mais confortável, pode ser uma boa escolha.
4. **Le Trésor**: É um serviço de

In [19]:
print("Testando a cadeia de Tradução...")
pergunta_traducao = "Como dizer 'onde fica o banheiro' em francês?"
resposta_traducao = translation_chain.invoke({"question": pergunta_traducao})
print(f"Pergunta: {pergunta_traducao}\nResposta:\n{resposta_traducao}")
print("="*80)

Testando a cadeia de Tradução...
Pergunta: Como dizer 'onde fica o banheiro' em francês?
Resposta:
A frase "Como dizer 'onde fica o banheiro' em francês?" em inglês é:

"How do you say 'where is the bathroom' in French?"

E a resposta é:

"Comment dit-on 'où est la salle de bain' en français?"

Se você quiser uma resposta mais simples e direta, pode usar:

"Où est la salle de bain?"

Isso significa "Onde fica o banheiro?" em francês.
